<a href="https://colab.research.google.com/github/quaneh/portfolio/blob/main/NeMo_Guardrails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NeMo Guardrails

In this notebook I will be investigating how to use NeMo Guardrails to create AI chatbots. I'll be following along with some tutorials from the wonderful James Briggs, please check out his videos on his chanel: https://www.youtube.com/@jamesbriggs/videos

In [1]:
!pip install -qU nemoguardrails openai langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 25.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.5/814.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.4/278.4 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.9 MB/s

In [2]:
from getpass import getpass

OPENAI_API_KEY = getpass()

··········


In [3]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

As I'm running this notebook in Colab, I'll import the config from a string. However, this config can be found in config/config.yml  and config/topics.co
These are our colang file and our config file.

In [23]:
yaml_content =  """
models:
- type: main
  engine: openai
  model: gpt-3.5-turbo-instruct
"""

colang_content = """
define user give name
    "My name is Plato"
    "I'm Aristotle"
    "I am Johnny"


define user greeting
    "hello"
    "hi"
    "what's up?"

define user ask politics
    "what are your political beliefs?"
    "thoughts on the president?"
    "left wing"
    "right wing"

define bot name greeting
    "Hey $name!"

define bot answer politics
    "I'm a personal assistant, I don't like to talk of politics."

define flow give name
    user give name
    $name = ...
    bot name greeting

define flow
    user greeting
    if $name
        bot name greeting
    else
        bot ask name

define flow politics
    user ask politics
    bot answer politics
    bot offer help
"""

In [24]:

from nemoguardrails import LLMRails, RailsConfig

config = RailsConfig.from_content(
    yaml_content=yaml_content,
    colang_content=colang_content
)

rails = LLMRails(config)

We'll start by just passing simple promts to our rails, to see how they react.

We can see that when we ask a political question, we are directed to the politics flow.

In [25]:
res = await rails.generate_async(prompt='hey there')
print(res)

Hey there! May I know your name?


In [26]:
res = await rails.generate_async(prompt='Who\'s the prime minister of ireland')
print(res)

I'm a personal assistant, I don't like to talk of politics.
I can still help you with other tasks though, just let me know!


Next we'll define some messages, and pass them to our rails.
This allows us to do more clever things, like passing message history, or some context.

In [31]:
messages = [
    {"role": "context", "content": ""},
    {"role": "user", "content": "Hey there!"}
]

In [32]:
res = await rails.generate_async(messages=messages)
res

{'role': 'assistant', 'content': "Hi there! What's your name?"}

In [33]:
messages += [
    res,
    {"role": "user", "content": "I'm Hugh"}
]

In [34]:
res = await rails.generate_async(messages=messages)
res

{'role': 'assistant', 'content': 'Hey Hugh!'}